# Neural network class with better functions

In this notebok, I am going to clean up some of code from the prior notebooks and wrap the whole thing up in a nice class structure. 

Additionally, I have added the ability to scale up to more neurons. Lastly, I demonstrate the network running some actually useful classification tasks using scikit's Iris and Breast cancer datasets. 

In [270]:
import numpy as np


class neuralnetwork(object):
    def __init__(self, sizes):
        np.random.seed(42)
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.zs = []
        self.activations = []
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        #print("Initial Biases: \n{}".format(self.biases))
        #print("Initial Weights: \n{}".format(self.weights))
    def reinitialize(self):
        self.zs = []
        self.activations = []
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]    
        #print("Initial Biases: \n{}".format(self.biases))
        #print("Initial Weights: \n{}".format(self.weights))
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    def cost_function(self, y, y_predicted):
        return .5*(y - y_predicted)**2
    def cost_function_deriv(self, y_predicted, y):
        #print("calculating cost_function_deriv: \ny_predicted is:\n{}\n and y is:\n{}\n".format(y_predicted, y))
        return y_predicted - y
    def activation_function(self, z):
        return self.sigmoid(z)
    def activation_function_deriv(self, z):
        return self.sigmoid(z)*(1-self.sigmoid(z))
    
    def feedforward(self, a):
        #print("-------in the feedforward-------------")
        self.zs = []
        self.activations = [a]
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, a)+b
            self.zs.append(z)
            a = self.activation_function(z)
            self.activations.append(a)
        #print("Observed Vectors:\n{}".format(self.zs))
        #print("Observed Activations:\n{}".format(self.activations))
        #print("feedforward returns: {}".format(a))
        #print("-----End feedforward-------\n")
        return a
    
    def backprop(self, y):
        #print("-------in the backprop-------------")
        #print("zs:\n{}\nactivations:\n{}\n".format(self.zs, self.activations))
        biasgrads = [np.zeros(b.shape) for b in self.biases]
        weightgrads = [np.zeros(w.shape) for w in self.weights]
        #print("The zeros:\nbiasgrads:\n{}\nweightgrads:\n{}".format(biasgrads, weightgrads))
        #print("Making the delta. \ncost_function_deriv is:\n{}\nactivation_function_deriv is:\n{}\n\n".format(self.cost_function_deriv(self.activations[-1], y), self.activation_function_deriv(self.zs[-1])))
        delta = self.cost_function_deriv(self.activations[-1], y) * self.activation_function_deriv(self.zs[-1])
        #print("delta:{}".format(delta))
        biasgrads[-1] = np.sum(delta)
        weightgrads[-1] = np.dot(delta, self.activations[-2].transpose())
        #print("after delta:\nbiasgrads:\n{}\nweightgrads:\n{}".format(biasgrads, weightgrads))
        #print("Now the for loop...")
        for l in range(2,self.num_layers):
            z = self.zs[-l]
            delta = np.dot(self.weights[-l+1].transpose(), delta) * self.activation_function_deriv(z)
            biasgrads[-l] = np.sum(delta)
            weightgrads[-l] = np.dot(delta, self.activations[-l-1].transpose())
        #print("Backprop returning:\nbias grad:{}\nweightgrads:\n{}".format(biasgrads, weightgrads))    
        #print("--------End Backprop--------------------\n")
        return biasgrads, weightgrads
    
    def train(self, X, y, learning_rate, epochs):
        self.reinitialize()
        #print("Bias defaults: {}".format(self.biases))
        #print("Weight defaults: {}\n\n".format(self.weights))  
        for epoch in range(epochs):
            
            #print("------------------------Running epoch {}--------------------------\n".format(epoch))
            biasgradients = [np.zeros(b.shape) for b in self.biases]
            weightgradients = [np.zeros(w.shape) for w in self.weights]   
            
            #print("Epoch Bias gradient default: {}".format(biasgradients))
            #print("Epoch Weight gradient default: {}\n".format(biasgradients))    
            
            #forward propagation
            output = self.feedforward(X.T)
            #print("Output: {}".format(output))
            
            #compute the cost
            cost = self.cost_function(y.T, output)
            #print("Calculated Cost: {}\n".format(cost))
            
            #get the gradients
            #print("Zs: {}".format(self.zs))
            #print("Activations: {}".format(self.activations))
                    
            delta_biasgradients, delta_weightgradients = self.backprop(y.T)
            
            #print("delta Bias gradients: {}".format(delta_biasgradients))
            #print("delta Weight gradients: {}\n\n".format(delta_weightgradients))
            
            biasgradients = [bgrad + dbgrad for bgrad, dbgrad in zip(biasgradients, delta_biasgradients)]
            weightgradients = [wgrad + dwgrad for wgrad, dwgrad in zip(weightgradients, delta_weightgradients)]
            
            #print("Bias gradients: {}".format(biasgradients))
            #print("Weight gradients: {}\n\n".format(weightgradients))
                      
            #update the weights and biases
            self.biases = [b - learning_rate * biasgrad for b, biasgrad in zip(self.biases, biasgradients)]
            self.weights = [w - learning_rate * weightgrad for w, weightgrad in zip(self.weights, weightgradients)]

            #print("Biases: {}".format(self.biases))
            #print("Weights: {}\n\n".format(self.weights))
            
            #print("----------------------------End Epoch {}----------------------------\n\n".format(epoch))
        #print("Final Cost: {}".format(cost))
        #print("Final Biases: {}".format(self.biases))
        #print("Final Weights: {}\n\n".format(self.weights))
    def predict(self, X):
        return self.feedforward(X.T)


In [271]:
net = neuralnetwork([1,1,1])

X = np.array([
    [0],
    [1]
])


y = np.array([
    [0],
    [1]
])
#set up the learning rate
lr = 0.2
#set up the number of epochs
e = 1

net.train(X, y, lr, e)

In [272]:
net = neuralnetwork([2,2,1])

X = np.array([
    [0,0],
    [0,1],
    [1,0],
    [1,1]
])


y = np.array([
    [0],
    [0],
    [0],
    [1],
])
#set up the learning rate
lr = 0.2
#set up the number of epochs
e = 1

net.train(X, y, lr, e)


In [273]:
net = neuralnetwork([3,2,1])


#Demonstrate significant weight on the first input variable, 
#and none on the other two. 
X = np.array([
    [0,0,0],
    [0,0,1],
    [0,1,0],
    [0,1,1],
    [1,0,0],
    [1,0,1],
    [1,1,0],
    [1,1,1],

])

#for this goal these all match the first input
y = np.array([
    [0],
    [0],
    [0],
    [0],
    [1],
    [1],
    [1],
    [1],
])
#set up the learning rate
lr = 0.20
#set up the number of epochs
e = 500

net.train(X, y, lr, e)

print("--------------------Predictions")

print("Should be 1: {}".format(net.predict(np.array([[1,0,0]]))))

print("Should be 0: {}".format(net.predict(np.array([[0,1,1]]))))

--------------------Predictions
Should be 1: [[ 0.93325716]]
Should be 0: [[ 0.07671716]]


In [274]:
import numpy  as np
from  sklearn import datasets

# Importing the dataset
bcancer = datasets.load_breast_cancer()
X = bcancer.data
y = bcancer.target

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)


# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

net = neuralnetwork([30,20,10,1])
#set up the learning rate
lr = 0.20
#set up the number of epochs
e = 500
net.train(X_train, y_train, lr, e)

print("Predicted:")
print(net.predict(X_test))
print("Actual:")
print(y_test)

Predicted:
[[  4.65426872e-05   9.99999567e-01   9.99999922e-01   9.99999897e-01
    9.99999924e-01   9.99999882e-01   9.99999588e-01   9.99999946e-01
    9.99999905e-01   9.99999947e-01   9.88579609e-01   9.99999654e-01
    9.99999954e-01   4.02615709e-01   7.11449704e-01   5.44290209e-05
    9.99965246e-01   4.48697725e-05   4.54146584e-05   4.56131847e-05
    6.00961003e-05   5.06977137e-05   9.99999635e-01   9.99999933e-01
    4.56235956e-05   9.99999931e-01   9.99999954e-01   8.46061082e-05
    9.99999922e-01   4.49945333e-05   9.99999959e-01   4.61650633e-05
    9.99568896e-01   4.79118878e-05   9.99999948e-01   5.12566012e-05
    9.99999268e-01   4.97206406e-05   9.99999586e-01   4.32018906e-05
    1.78340102e-03   9.99999955e-01   2.93014069e-04   9.99999949e-01
    9.99995450e-01   4.50643844e-05   9.99999949e-01   9.99999895e-01
    9.99999952e-01   4.71270393e-05   5.90678740e-05   1.44708615e-04
    4.40199839e-05   9.99999792e-01   9.99999957e-01   9.99999761e-01
    9.999

In [276]:
import numpy  as np
from  sklearn import datasets
from sklearn.preprocessing import OneHotEncoder

# Importing the dataset
dataset = datasets.load_iris()
X = dataset.data
y = dataset.target

#encode the result set
enc = OneHotEncoder()
y_encoded = enc.fit_transform(np.expand_dims(y, axis=1))
y = y_encoded.toarray()

#split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

#scale
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



#neural network

net = neuralnetwork([4,8,8,3])
#set up the learning rate
lr = 0.20
#set up the number of epochs
e = 500
net.train(X_train, y_train, lr, e)

print("Predicted:")
print(net.predict(X_test).T)
print("Actual:")
print(y_test)


Predicted:
[[  1.29207056e-03   2.77715322e-03   9.94256739e-01]
 [  1.87267834e-02   9.88722909e-01   7.46471594e-03]
 [  9.88308674e-01   5.56403663e-03   1.14001310e-03]
 [  9.58396305e-04   4.80031331e-03   9.93180454e-01]
 [  9.87810959e-01   6.29659740e-03   1.12952738e-03]
 [  1.34985850e-03   2.50497315e-03   9.94573941e-01]
 [  9.87994841e-01   5.99134853e-03   1.13697935e-03]
 [  1.87483209e-02   9.85008084e-01   9.30218495e-03]
 [  1.77269791e-02   9.83652353e-01   1.04996555e-02]
 [  2.00493753e-02   9.87506197e-01   7.56829367e-03]
 [  5.53151338e-04   2.73541559e-02   9.84165489e-01]
 [  1.93836753e-02   9.86111541e-01   8.49052849e-03]
 [  1.68971409e-02   9.82805721e-01   1.14664426e-02]
 [  1.76347965e-02   9.83073577e-01   1.08277126e-02]
 [  1.49778999e-02   9.77567535e-01   1.57741416e-02]
 [  9.87271950e-01   7.95836149e-03   1.08021975e-03]
 [  1.64976126e-02   9.80972170e-01   1.26584098e-02]
 [  1.60155110e-02   9.81543423e-01   1.27520841e-02]
 [  9.83745998e-0